In [62]:
import re

IMG-20241028-WA0020.jpg
IMG-20241028-WA0021.jpg
IMG-20241028-WA0022.jpg
IMG-20241028-WA0023.jpg
IMG-20241028-WA0024.jpg
IMG-20241028-WA0025.jpg
IMG-20241028-WA0027.jpg
IMG-20241028-WA0030.jpg
IMG-20241028-WA0031.jpg
IMG-20241028-WA0033.jpg
IMG-20241028-WA0035.jpg
IMG-20241028-WA0037.jpg
IMG-20241028-WA0039.jpg
IMG-20241028-WA0041.jpg
IMG-20241028-WA0043.jpg
IMG-20241028-WA0045.jpg
IMG-20241028-WA0049.jpg
IMG-20241028-WA0050.jpg
IMG-20241028-WA0056.jpg
IMG-20241028-WA0052.jpg
IMG-20241028-WA0054.jpg
IMG-20241028-WA0059.jpg
IMG-20241028-WA0061.jpg
IMG-20241028-WA0064.jpg
IMG-20241028-WA0065.jpg
IMG-20241028-WA0068.jpg
IMG-20241028-WA0069.jpg
IMG-20241028-WA0072.jpg
IMG-20241028-WA0073.jpg
IMG-20241028-WA0076.jpg
IMG-20241028-WA0077.jpg
IMG-20241028-WA0080.jpg
IMG-20241028-WA0081.jpg
IMG-20241028-WA0083.jpg
IMG-20241028-WA0085.jpg
IMG-20241028-WA0088.jpg
IMG-20241028-WA0089.jpg
IMG-20241028-WA0092.jpg
IMG-20241028-WA0093.jpg
IMG-20241028-WA0096.jpg
IMG-20241028-WA0097.jpg
IMG-20241028-WA0

In [85]:
#puxando nome e artista do txt
with open('txt/Conversa do WhatsApp com Pai.txt', 'r') as cvs:
    cvs_lines = cvs.readlines();

padrao_imagem = r'IMG-\d{8}-WA\d{4}\.jpg'

with open('txt/discos.txt', 'w') as albuns_txt:
    for cvs_line in cvs_lines:
        imagens = re.findall(padrao_imagem, cvs_line)
        for imagem in imagens:
            lista_imagens = []
            lista_imagens.append(imagem)
            capa = f"capa: {imagem}"
            print(capa)
            if len(lista_imagens) > 2:
                lista_imagens.clear()
        if '28/10/2024' not in cvs_line:
            cvs_line = cvs_line.strip()
            print(cvs_line + '\n')
            #albuns_txt.write(cvs_line + '\n')

capa: IMG-20241028-WA0020.jpg
Guns n' roses, Live ?!*@ Like a Suicide

capa: IMG-20241028-WA0021.jpg
capa: IMG-20241028-WA0022.jpg
SP Metal Vol. 2

capa: IMG-20241028-WA0023.jpg
capa: IMG-20241028-WA0024.jpg
A-ha, Burning high and low

capa: IMG-20241028-WA0025.jpg
capa: IMG-20241028-WA0027.jpg
Motörhead, Ace of spades

capa: IMG-20241028-WA0030.jpg
David Bowie, Labyrinth

capa: IMG-20241028-WA0031.jpg
capa: IMG-20241028-WA0033.jpg
Nazareth, Bazamanaz

capa: IMG-20241028-WA0035.jpg
capa: IMG-20241028-WA0037.jpg
Venom, Welcome to hell

capa: IMG-20241028-WA0039.jpg
capa: IMG-20241028-WA0041.jpg
Judas Priest, Unleashed in the east

capa: IMG-20241028-WA0043.jpg
capa: IMG-20241028-WA0045.jpg
Saxon, Crusader

capa: IMG-20241028-WA0049.jpg
capa: IMG-20241028-WA0050.jpg
Van halen, 1984

capa: IMG-20241028-WA0056.jpg
capa: IMG-20241028-WA0052.jpg
Running wild, Gates to purgatory

capa: IMG-20241028-WA0054.jpg
capa: IMG-20241028-WA0059.jpg
Accept, Kaizoku-ban

capa: IMG-20241028-WA0061.jpg
cap

In [54]:
with open('txt/discos.txt', 'r') as discos:
    discos = discos.readlines()
    #print(discos)

for linha in discos:
    linha = linha.split(',')
    artista = linha[0]
    if len(linha) < 2:
        nome = linha[0]
    else:
        nome = linha[1]
    artista = artista.strip()
    nome = nome.strip()
    print("Artista: "+artista)
    print("Nome: "+nome)

Artista: Guns n' roses
Nome: Live ?!*@ Like a Suicide
Artista: SP Metal Vol. 2
Nome: SP Metal Vol. 2
Artista: A-ha
Nome: Burning high and low
Artista: Motörhead
Nome: Ace of spades
Artista: David Bowie
Nome: Labyrinth
Artista: Nazareth
Nome: Bazamanaz
Artista: Venom
Nome: Welcome to hell
Artista: Judas Priest
Nome: Unleashed in the east
Artista: Saxon
Nome: Crusader
Artista: Van halen
Nome: 1984
Artista: Running wild
Nome: Gates to purgatory
Artista: Accept
Nome: Kaizoku-ban
Artista: The alan parsons project
Nome: Vulture culture
Artista: Bagga's guru
Nome: Pirata
Artista: Abba
Nome: Golden Hits
Artista: 12 super tops
Nome: 12 super tops
Artista: Spin doctors
Nome: Turn it upside down
Artista: Bread
Nome: Anthology of bread
Artista: The rolling stones
Nome: Rolled gold
Artista: Bob marley and The wailers
Nome: 12 reggae hits
Artista: George michael and queen with lisa stanfield
Nome: Five live
Artista: Rainbow
Nome: Straight between the eyes
Artista: The 80s 4 lp's
Nome: The 80s 4 lp's